In [44]:
import h5py
import pandas as pd
import numpy as np

In [45]:
def hdf5_to_dict(my_dict, h5py_file):
    """
    递归转换 h5py 的 HDF5 数据集为 python 字典，同时解析 HDF5 object references
    """
    result = {}
    for key, item in my_dict.items():
        if isinstance(item, h5py.Reference):
            # 如果这是一个 HDF5 object reference，获取引用的实际对象
            actual_item = h5py_file.get(item)
            if isinstance(actual_item, h5py.Dataset):
                result[key] = actual_item[:]
            elif isinstance(actual_item, h5py.Group):
                result[key] = hdf5_to_dict(actual_item, h5py_file)
        elif isinstance(item, h5py.Dataset):
            # 如果这是一个数据集，直接获取数据
            result[key] = item[:]
        elif isinstance(item, h5py.Group):
            # 如果这是一个 group，递归地转换为字典
            result[key] = hdf5_to_dict(item, h5py_file)
    return result

In [46]:
def replace_empty_cells(data):
    """
    遍历嵌套列表（对应于嵌套 cell 数组），并替换所有空列表为 'N/A'
    """
    if isinstance(data, list):
        if not data:  # 如果列表为空
            return 'N/A'
        else:  # 如果列表不为空
            return [replace_empty_cells(item) for item in data]
    else:  # 如果数据不是列表
        return data

In [47]:
f = h5py.File('/Users/xichenhao/Desktop/course of Leeds/MSc project/Project/Kato_data/AVA_HisCl.mat', 'r')
data_dict = hdf5_to_dict(f,f)

In [48]:
#确定哪些值的多维的
for key, value in data_dict['AVA_HisCl'].items():
    print(f"{key}: {np.shape(value)}")

IDs: (5, 1)
States: (5, 1)
States_key: (5, 1)
dataset: (5, 1)
fps: (5, 1)
timeVectorSeconds: (5, 1)
traces: (5, 1)
tracesDif: (5, 1)
traces_raw: (5, 1)


In [49]:
data_dict = {f"{key}_{i}": value[i, :] for key, value in data_dict['AVA_HisCl'].items() for i in range(value.shape[0])}

In [50]:
df = pd.DataFrame(data_dict)
df = df.applymap(replace_empty_cells)
df.to_excel('output.xlsx', index=False)